# 处理本地文件


本地仓库的目录下每一个文件都不外乎处于两种状态：已追踪的 (tracked) 或未追踪的 (untracked)。
- 已追踪的文件包括上一次 git 所记录的快照中的文件，以及新的暂存文件；它们可能是未编辑的 (unmodified)、已编辑的 (modified) 或已暂存的 (staged) —— Git 会将编辑后的文件标记为已编辑状态，此时我们可以根据需要，将若干修改过的文件移入暂存区，远程提交时只会提交已暂存的修改；
- 工作目录下除已跟踪文件外的其它所有文件都属于未追踪文件，这些文件既不存在于快照记录中，也没有被放入暂存区；





![](https://git-scm.com/book/en/v2/images/lifecycle.png)







运行`git status`即可查看当前仓库中所有文件的状态

## 1. 追踪文件

使用命令`git add <file>`可以对一个新创建的文件进行追踪，其中 `<file>` 可以是文件或目录的路径，对于目录路径的情况，该命令会递归地追踪目录下的所有文件。此外，也可以一次性地传递`--all`或`-A`参数来对所有文件进行追踪

In [ ]:
! git add --all

In [ ]:
# check the status
! git status

## 2. 暂存文件

对于那些修改后的文件，若希望在远程推送时将其纳入推送的文件列表中，还需要将该文件放入暂存区。暂存文件的命令依旧是 `git add` —— 该命令既可以追踪新文件，也可以将已跟踪的文件放到暂存区，还有将合并时冲突的文件标记为已解决状态等其他功能；

In [ ]:
! git add --all
! git status

利用`git reset HEAD <FILE>`或`git restore --staged <FILE>`可以取消对某个文件的暂存。这里需要注意，`git reset`命令可能会造成较为危险的结果，尤其在使用`--hard`参数时。更多有关`reset`的信息请参见 [Reset Demystified](https://git-scm.com/book/en/v2/ch00/_git_reset)。

In [ ]:
! git restore --staged ../
! git status

## 3. 提交文件

`git commit`命令可以将**暂存区的**文件进行提交，输入该命令后会弹出指定的文本编辑器，以供输入提交说明。其中编辑器是由 Shell 的环境变量`EDITOR`指定的，一般为 vim 或 emacs，当然也可以按照 [01_Git_Setup](http://localhost:8888/notebooks/Help_Viewer_Git/01_Git_Setup.ipynb) 一节中说明的方式进行设置。

编辑器会显示类似下面的文本信息（这里显示的是 vim 中的内容）：


![](./img/git_commit_demo.png)




可以看出，默认的提交消息包含最后一次运行`git status`的输出，文件第一行用于输入提交说明，后面的注释可以选择性的保留或删除。若需要将文件更改的具体内容`diff`（参见下一节）也添加在提交说明里，可以通过附加`-v`参数来实现。

此外，也可以通过`-m`参数将提交信息附加在`git commit`命令后，提交信息需用`""`标注：

In [ ]:
! git commit -m "add some notes for Git"

需要说明的是，每一次运行提交操作，都是对项目进行一次快照记录，用户可以随时在需要的时候回到之前的版本，或与之前的版本进行比较。

**略过暂存操作**

尽管使用暂存区域可以处理提交细节，但有时候这么做略显繁琐。Git 提供了一个略过暂存操作的方式，通过`git commit -a`便可以自动把所有已经跟踪过的文件暂存起来一并提交，从而跳过`git add`步骤，随后添加提交信息的过程与上面所述相同。例如`git commit -a -m "modified README"`，但还是需要留心，因为这个命令可能会将不需要的文件添加到提交中；


**重新提交**

如果用户不小心提前进行了提交，或者输错了提交信息，可以通过`git commit --amend`命令来重新提交，例如
```bash
git commit -m 'Initial commit'
git add forgotten_file
git commit --amend
```
需要说明的是，这个过程其实并非对上一次的提交进行修改，而是**用一个新的提交来完全替换之前的记录，以前的提交也不会显示在存储库历史记录中。**

这里建议只对那些局部的、没有被推送到其他分支的提交进行修改 —— 先前推送的提交和强制推送分支可能会给你的合作者带来问题，更多信息可以参阅[变基的风险](https://git-scm.com/book/en/v2/ch00/_rebase_peril)

## 4. gitignore

对于一些无需纳入 Git 管理的文件，例如自动生成的文件、数据库等，可以创建一个名为`.gitignore`的文件，在其中列出要忽略的文件的模式。`.gitignore`的格式规范如下：
- 所有空行或者以`#`开头的行都会被 Git 忽略；
- 可以使用标准的 glob 模式匹配，它会递归地应用在整个工作区中；这里 glob 模式指 shell 的简化正则表达式，例如
    - `*`匹配零个或多个的任意字符；
    - `[abc]`匹配任何一个列在方括号中的字符；
    - `[a-9]`表示所有在这两个字符范围内的字符都可以匹配，
    - `?`匹配任意单个字符；
    - 使用`**`表示匹配任意中间目录，例如`a/**/z`可以匹配`a/z`、`a/b/c/z`
    - ……
- 以`/`开头防止递归；
- 以`/`结尾指定目录；
- 要忽略指定模式以外的文件或目录，可以在模式前加上`!`取反

例如下面的示例：
```bash
*.[oa]          # 忽略所有以`.o`或`.a`结尾的文件
*.txt           # 忽略所有 .txt 后缀的文件
!src.a          # 忽略除 src.a 外的其他文件
/todo           # 仅忽略项目根目录下的 todo 文件，不包括 src/todo
build/          # 忽略 build/目录下的所有文件，过滤整个build文件夹；
doc/*.txt       # 忽略doc目录下所有 .txt 后缀的文件，但不包括doc子目录的 .txt 的文件
 
bin/:           # 忽略当前路径下的 bin 文件夹，该文件夹下的所有内容都会被忽略，不忽略 bin 文件
/bin:           # 忽略根目录下的 bin 文件
/*.c:           # 忽略 cat.c，不忽略 build/cat.c
debug/*.obj:    # 忽略debug/io.obj，不忽略 debug/common/io.obj和tools/debug/io.obj
**/foo:         # 忽略/foo, a/foo, a/b/foo等
a/**/b:         # 忽略a/b, a/x/b, a/x/y/b等
!/bin/run.sh    # 不忽略bin目录下的run.sh文件
*.log:          # 忽略所有 .log 文件
config.js:      # 忽略当前路径的 config.js 文件
/mtk/           # 忽略整个文件夹
*.zip           # 忽略所有.zip文件
/mtk/do.c       # 忽略某个具体文件
```

此外，可能还需要忽略文件包括`log`，`tmp`、`pid`目录，以及自动生成的文档等等。这里建议用户要养成一开始就为新仓库设置`.gitignore`文件的习惯，以免将来误提交这类无用的文件。

GitHub 有个针对数十种项目及语言的 .gitignore 文件列表，可参见[相关文档](https://github.com/github/gitignore) ；最简单的情况下，一个仓库只在根目录下有`.gitignore`文件，它递归地应用到整个仓库中；此外，子目录下也可以有额外的`.gitignore`文件，其只作用于它所在的目录中；多`.gitignore`文件的具体细节请参见官方文档；

需要说明的是，`.gitignore`只能忽略那些原来没有被追踪的文件，如果某些文件已经被纳入了版本管理中，此时修改`.gitignore`是无效的。因此需要先使 Git 停止对该文件的追踪（见下），再在`.gitignore`文件添加忽略规则。

## 5. 移除文件

在 Git 中，`git rm -f <FILE>`可以实现停止对该文件的追踪，同时从工作目录中删除的效果，其中`<FILE>`可以使用 glob 语法，例如`git rm -f log/\*.log`，

如果单纯手动、或从 cmd 命令行中删除文件，在运行`git status`时仍会在未暂存列表`Changes not staged for commit`中看到之前删除的文件：

In [ ]:
! echo test>./tmp/test.txt
! git add ./tmp/test.txt
! del tmp\\test.txt
! git status

使用`git rm -f <FILE>`则会直接移除对该文件的追踪，下一次提交时，该文件就不会被纳入版本管理了。注意！如果被删除文件所在的目录里只有该文件时，git 会连带这个被删除文件的父文件夹一起删除掉，例如`tmp`文件夹里只有`tmp.txt`文件，则执行`git rm -f tmp/tmp.txt`后，`tmp`文件夹也会一并被删除。下面是删除`tmp/test.txt`文件的示例：

In [ ]:
! echo test>./tmp/test.txt
! git add ./tmp/test.txt
! git rm -f tmp/test.txt
! git status

如果希望把文件从暂存区域移除，但仍保留在当前工作目录中，可以通过附加`--cached`参数来实现，例如：

In [ ]:
! echo test>./tmp/test_kpt.txt
! git add ./tmp/test_kpt.txt
! git rm -f --cached ./tmp/test_kpt.txt 
! git status

对于删除文件的情况，需要附加`-r`命令来实现递归

## 6. 重命名文件

在 Git 中，重命名是通过`git mv`命令实现的，例如`git mv README ReadMe`。这个命令确实看起来很令人疑惑，因为 Git 与其它的 VCS 不同，它并不显式跟踪文件移动操作；进而如果在 Git 中重命名了某个文件，Git 中存储的元数据并不会体现出这是一次重命名操作，不过 Git 还是可以根据推断出究竟发生了什么，例如`git mv`相当于运行了下面三条命令：
```bash
$ mv README.md README
$ git rm README.md
$ git add README
```
如此分开操作，Git 也会意识到这是一次重命名，所以不管何种方式结果都一样；在用其他工具批处理重命名时，要记得在提交前删除旧的文件名，再添加新的文件名；

## 7. 撤销对文件的更改

利用`git checkout -- <file>`或`git restore <FILE>`可以将文件还原成上次提交时的状态，这意味着对该文件所做的任何本地更改都将消失，因此这个命令一定要谨慎使用。

如果希望保留对该文件所做的更改，但依旧需要将其恢复至之前的样子，可以参见 [Git Branching](https://git-scm.com/book/en/v2/ch00/ch03-git-branching)中的指导来将该文件进行分支并隐藏。

请注意，在 Git 中提交的几乎任何内容都可以被恢复，即使是分支上被删除或被`--amend`提交所覆盖的提交也可以恢复（请参见 [Data Recovery](https://git-scm.com/book/en/v2/ch00/_data_recovery)）然而对于那些从未别提交的文件，可能无论怎么样也无法再找到了。